In [2]:
import os
import h5py
import numpy as np
import pandas as pd
from rich.progress import track
from scipy.spatial.distance import cosine
from itertools import product
from rich.progress import track

rng = np.random.default_rng(0)

In [4]:
with h5py.File("/media/scratch/sageev-midi/20250320/specdiff.h5", "r") as hdf:
    print(hdf.keys())
    print(hdf["embeddings"])
    print(str(hdf["filenames"][0], "utf-8"))
    print(hdf["embeddings"][:5])

<KeysViewHDF5 ['embeddings', 'filenames']>


KeyError: 'Unable to synchronously open object (invalid dataset size, likely file corruption)'

In [6]:
# p_h5 = "../data/tables/20250110/specdiff.h5"
# with h5py.File(p_h5, "r") as hdf_in:
#     with h5py.File("../data/tables/20250110/specdiff_new.h5", "w") as hdf_out:
#         new_names = hdf_out.create_dataset(
#             "filenames",
#             (len(hdf_in["filenames"]), 1),
#             dtype=h5py.string_dtype(encoding="utf-8"),
#             fillvalue="",
#         )
#         embeddings = hdf_out.create_dataset(
#             "embeddings", (len(hdf_in["embeddings"]), 768)
#         )

#         for i, name in track(enumerate(hdf_in["filenames"])):
#             new_names[i] = str(name[0][:-7], "utf-8")
#             embeddings[i] = hdf_in["embeddings"][i]

Output()

In [13]:
with h5py.File("../data/tables/20250110/specdiff.h5", "r") as hdf:
    print(hdf["filenames"][:5])
    print(hdf["embeddings"][:5])
with h5py.File("../data/tables/20250110/pitch_histogram.h5", "r") as hdf:
    print(hdf["filenames"][:5])
    print(hdf["histograms"][:5])

[[b'20231220-080-01_0000-0005_t00s00']
 [b'20231220-080-01_0000-0005_t00s01']
 [b'20231220-080-01_0000-0005_t00s02']
 [b'20231220-080-01_0000-0005_t00s03']
 [b'20231220-080-01_0000-0005_t00s04']]
[[0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]
 [0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]
 [0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]
 [0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]
 [0.         0.         0.18383147 0.         0.         0.
  0.         0.81616855 0.         0.         0.         0.        ]]


In [5]:
# verify hdf5 file
p_h5 = "../data/tables/20250110/specdiff.h5"
with h5py.File(p_h5, "r") as hdf:
    print(f"{len(hdf.keys())} datasets in file")
    new_names = hdf.create_dataset("filenames")
    df = pd.DataFrame(
        list([e] for e in hdf["embeddings"]),
        index=[str(name[0][:-7], "utf-8") for name in hdf["filenames"]],
        columns=["embeddings"],
    )
df.head()
# df.to_hdf("../data/tables/20250110/specdiff_new.h5", key="embeddings")

2 datasets in file


,embeddings
20231220-080-01_0000-0005_t00s00,"[-0.053769037, -0.13694727, 0.061716847, 0.084..."
20231220-080-01_0000-0005_t00s01,"[-0.038331196, -0.13422038, 0.0947322, 0.09812..."
20231220-080-01_0000-0005_t00s02,"[-0.05655879, -0.15524468, 0.057764757, 0.0674..."
20231220-080-01_0000-0005_t00s03,"[-0.043163337, -0.13563132, 0.09134518, 0.0746..."
20231220-080-01_0000-0005_t00s04,"[-0.057311468, -0.15267283, 0.07441097, 0.0672..."


In [ ]:
# load prompt dataset
p_test = "../data/datasets/test/dataset samples"
transformations = [f"t{t:02d}s{s:02d}" for t, s in product(range(12), range(8))]
q_files = [f"{f[:-4]}_{rng.choice(transformations)}" for f in os.listdir(p_test)]
q_files[:3]

In [ ]:
# load prompt embeddings
q_embeddings = [embedding_dataset[key] for key in q_files]
q_embeddings[0]

In [ ]:
embedding_dataset.values()

In [ ]:
# find best and worst n matches for prompts
n_matches = 3
indices = []  # To store the indices of the best and worst matches
for q_emb in q_embeddings:
    similarities = []
    for idx, dataset in enumerate(embedding_dataset.values()):
        sim = 1 - cosine(q_emb, dataset)
        similarities.append((sim, idx))

    similarities.sort(key=lambda x: x[0], reverse=True)  # Highest first
    best_indices = [idx for _, idx in similarities[:n_matches]]  # Top 3
    worst_indices = [idx for _, idx in similarities[-n_matches:]]  # Bottom 3

    indices.append((best_indices, worst_indices))

In [ ]:
embedding_dataset.close()